# End-to-end Learning for Early Classification of Time Series (ELECTS)

## Train and predict with an RNN trained with ELECTS

Early predictions with the RNN model.

Please run `python train.py` first to generate `snapshots/model.pth`

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 12.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 32.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 KB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 79.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 66.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 82.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.8 MB/s eta 0:00:00:00:0100:01
   

In [1]:
import subprocess

# Start the Visdom server as a background process
subprocess.Popen(['python', '-m', 'visdom.server', "-port", "8097"])


<Popen: returncode: None args: ['python', '-m', 'visdom.server', '-port', '8...>

Checking for scripts.
It's Alive!


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/go98kef/.virtualenvs/venv/lib/python3.10/site-packages/visdom/server/__main__.py", line 16, in <module>
    download_scripts_and_run()
  File "/home/go98kef/.virtualenvs/venv/lib/python3.10/site-packages/visdom/server/run_server.py", line 236, in download_scripts_and_run
    main()
  File "/home/go98kef/.virtualenvs/venv/lib/python3.10/site-packages/visdom/server/run_server.py", line 220, in main
    start_server(
  File "/home/go98kef/.virtualenvs/venv/lib/python3.10/site-packages/visdom/server/run_server.py", line 55, in start_server
    app.listen(port, max_buffer_size=1024**3)
  File "/home/go98kef/.virtualenvs/venv/lib/python3.10/site-packages/tornado/web.py", line 2211, in listen
    server.listen(
  File "/home/go98kef/.virtua

In [4]:
!python train.py --snapshot snapshots/bugsense_model.pth

^C
Traceback (most recent call last):
  File "/media/storage/users/go98kef/ELECTS/ELECTS/ELECTS_early_time_series/train.py", line 1, in <module>
    from data import BavarianCrops, BugSenseData
  File "/media/storage/users/go98kef/ELECTS/ELECTS/ELECTS_early_time_series/data/__init__.py", line 1, in <module>
    from .bavariancrops import BavarianCrops
  File "/media/storage/users/go98kef/ELECTS/ELECTS/ELECTS_early_time_series/data/bavariancrops.py", line 1, in <module>
    import torch
  File "/home/go98kef/.virtualenvs/venv/lib/python3.10/site-packages/torch/__init__.py", line 367, in <module>
    from torch._C import *  # noqa: F403
  File "<frozen importlib._bootstrap>", line 216, in _lock_unlock_module
KeyboardInterrupt


## Import dependencies

In [12]:
import torch
from data import BugSenseData
import matplotlib.pyplot as plt
from earlyrnn import EarlyRNN
import seaborn as sns
import numpy as np
import os
CLASSES = ['ecoli', 'efaecalis/kpneumoniae', 'ssaprophyticus/ehormaechei', 
           'paeruginosa/pmirabilis', 'saureus', 'sterile']

In [13]:
script_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
print(script_dir)
root_dir = os.path.join(script_dir,  "BugSenseData")
ds = BugSenseData(root_dir=root_dir, partition="eval")
model = EarlyRNN()
model.eval()
model.load_state_dict(torch.load("snapshots/bugsense_model.pth"))

/media/storage/users/go98kef


/tmp/ipykernel_2036542/1377693546.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("snapshots/bugsense_model.pth"))


<All keys matched successfully>

## Take time series sample from dataset

The dataset contains Sentinel 2 satellite time series of Bavarian fields in 2018.
Each line represents spectral reflectances in one of 13 bands acquired by the satellite over the year.

In [14]:
# change this to take different samples
index = 3

X,y = ds[index]
print(y)

fig, ax = plt.subplots(figsize=(12,6))
ax.plot(X)
ax.set_title(CLASSES[y])
sns.despine(offset=0.2)

ValueError: only one element tensors can be converted to Python scalars

## Predict with the model

In [15]:
# change this T parameter to simulate shorter time series
T = 80

log_class_probabilities, probabilitiy_stopping = model(X[:T].unsqueeze(0))

# visualize
fig, axs = plt.subplots(2,1, figsize=(16,6))
ax = axs[0]
ax.plot(log_class_probabilities[0].exp().detach().numpy())
ax.set_ylim(0,1)
ax.legend(CLASSES, loc=1)
ax.set_title(f"classification probabilities (correct class = {CLASSES[y[0]]})")
ax = axs[1]
ax.bar(x = np.arange(probabilitiy_stopping.shape[1]), height = probabilitiy_stopping[0].detach().numpy())
ax.set_title("stopping probability")
ax.set_ylim(0,1)
sns.despine(offset=0.2)

NameError: name 'X' is not defined

INFO:tornado.access:200 POST /env/main (::1) 1.04ms
INFO:tornado.access:101 GET /vis_socket (::1) 0.61ms
INFO:root:Opened visdom socket from ip: ::1
INFO:tornado.access:200 POST /env/main (::1) 0.53ms
INFO:tornado.access:101 GET /vis_socket (::1) 0.47ms
INFO:root:Opened visdom socket from ip: ::1
INFO:tornado.access:200 POST /env/main (::1) 1.07ms
INFO:tornado.access:101 GET /vis_socket (::1) 0.58ms
INFO:root:Opened visdom socket from ip: ::1
INFO:tornado.access:200 POST /env/main (::1) 0.58ms
INFO:tornado.access:101 GET /vis_socket (::1) 0.47ms
INFO:root:Opened visdom socket from ip: ::1
INFO:tornado.access:200 POST /env/main (::1) 0.61ms
INFO:tornado.access:101 GET /vis_socket (::1) 0.48ms
INFO:root:Opened visdom socket from ip: ::1
INFO:tornado.access:200 POST /env/main (::1) 0.59ms
INFO:tornado.access:101 GET /vis_socket (::1) 0.46ms
INFO:root:Opened visdom socket from ip: ::1
INFO:tornado.access:200 POST /env/main (::1) 0.60ms
INFO:tornado.access:101 GET /vis_socket (::1) 0.46ms
I